In [1]:
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

2023-05-26 12:38:48.739871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-26 12:38:48.910209: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-26 12:38:48.910775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 12:38:50.051602: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!ls

In [ ]:
!python model_main_tf2.py --model_dir=output/ --pipeline_config_path=ssd_mobilenet_v2_ft/pipeline.config

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir src/output/ --output_directory src/ssd_mobilenet_v2_ft2

In [2]:
PATH_TO_MODEL_DIR = 'efficientnet512_ft/'
PATH_TO_LABELS = '/home/irizqy/ml_ws/bangkit-ws/data/label_map.pbtxt'

In [3]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

2023-05-26 12:38:54.090121: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 12:38:54.090526: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Done! Took 18.307851552963257 seconds


In [4]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [17]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os


rand_im = np.random.choice(os.listdir('/home/irizqy/ml_ws/bangkit-ws/data/test'), 3)
rand_im

BASED_PATH = '/home/irizqy/ml_ws/bangkit-ws/data/test'
IMAGE_PATHS = [os.path.join(BASED_PATH, im) for im in rand_im]
IMAGE_PATHS

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
count = 0

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)
    
    if image_np.shape[2] == 4:
        image_np = image_np[:, :, :3]

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # print(detections)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.35,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    im = Image.fromarray(image_np_with_detections)
    im.save(f"detected-logo2{count}.jpeg")
    count+=1
    print('Done')
    plt.show()

# sphinx_gallery_thumbnail_number = 2

Running inference for /home/irizqy/ml_ws/bangkit-ws/data/test/7_20230510_161041.jpg... defaultdict(<class 'str'>, {})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/test/17_20230507_092258.jpg... defaultdict(<class 'str'>, {(0.37275806069374084, 0.12145353853702545, 0.5225242376327515, 0.3400805592536926): 'Chartreuse', (0.36121851205825806, 0.35893332958221436, 0.5373385548591614, 0.8045828342437744): 'Aqua', (0.646271288394928, 0.28119826316833496, 0.7014930844306946, 0.42326074838638306): 'Aqua'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/test/31_20230510_161931.jpg... defaultdict(<class 'str'>, {(0.13620814681053162, 0.3898668587207794, 0.3163815140724182, 0.6464215517044067): 'Chartreuse'})
Done


In [29]:
import cv2 as cv

In [43]:
width, height, _ = image_np.shape

In [15]:
print(detections['detection_scores'])

[0.8888465  0.7940105  0.5593475  0.35204446 0.3131805  0.20242946
 0.20069684 0.15278015 0.14918803 0.13627309 0.12885123 0.12286025
 0.12188619 0.10864077 0.10621609 0.09073407 0.08411635 0.08162455
 0.081135   0.07932451 0.07764754 0.07757568 0.07507224 0.07210369
 0.07101718 0.06686036 0.06514898 0.06316725 0.06234721 0.05784463
 0.05766162 0.05717769 0.05652346 0.05472551 0.05293413 0.05291624
 0.05270847 0.05181587 0.04982031 0.0494574  0.04910188 0.04868054
 0.04734084 0.04691124 0.04607559 0.04591864 0.04591481 0.04541697
 0.04508835 0.04457164 0.04418835 0.04366131 0.04310258 0.04210387
 0.04198137 0.04126276 0.04100103 0.04066778 0.04019002 0.03998356
 0.03944269 0.03932796 0.03872728 0.03852287 0.03847083 0.03838234
 0.03831089 0.03799472 0.03677043 0.03671165 0.0355898  0.03538756
 0.03533013 0.03506214 0.03430734 0.03410344 0.03400818 0.0334891
 0.03273807 0.03230138 0.03210539 0.03207373 0.03169188 0.03165837
 0.03160166 0.03134772 0.03094308 0.03072072 0.03054597 0.03052

raw_detection_boxes <br>
detection_scores<br>
detection_boxes<br>
detection_anchor_indices<br>
raw_detection_scores<br>
detection_classes<br>
detection_multiclass_scores<br>
num_detections

In [44]:
ymin, xmin, ymax, xmax = detections['detection_boxes'][0]
(left, right, top, bottom) = (xmin*width, xmax*width, ymin*height, ymax*height)
print(int(left), int(right), int(top), int(bottom))

137 287 29 78


In [40]:
im = cv.rectangle(image_np, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 3)
Image.fromarray(im).save('test.jpg')

In [45]:
cropped_im = image_np[int(top):int(bottom), int(left):int(right)]
Image.fromarray(cropped_im).save('cropped-logo.jpg')
